In [ ]:
# imports
import imageio.v2 as imageio
from PIL import Image
from app import S3
import datetime
import fastdup
import os
import pprint as pp

bucket: str = "hydrahi4ai"
s3_img_path: str = "trash/fastdup/address_files_s3.txt"
local_img_path: str = "trash/fastdup/tmp/" + s3_img_path.split(bucket)[-1]
s3 = S3(bucket)

In [ ]:
def add_metadata(img_path: str = None, metadata: dict = None) -> str:
    
    if not os.path.exists(img_path):
        print(img_path)
        raise ValueError("La Dirección proporcionada no existe o esta mal escrita.")
    
    if not metadata:
        # Si el usario no define una metadata, nosotros agregamos
        metadata = {
            "msm": "Esta imagen no contenia Metadata",
            "repair_day": datetime.date.today()
        }
    elif not isinstance(metadata, dict):
        raise ValueError(f"El atributo de Metadata debe ser de tipo dict, no de tipo {type(metadata).__name__}")
    
    try:
        file_name = os.path.basename(img_path)
        
        image = imageio.imread(img_path)
        
        # re-escribimos la imagen con la metadata agregada
        imageio.imwrite(img_path, image, **metadata)
        print(f"Se acaba de agregar la metadata en el archivo {file_name}")
        
    except TypeError as e:
        print(e)
        
    finally:
        return img_path

In [ ]:
fd = fastdup.create("trash/test/")

# analisamos las imagenes del s3 para detectar las images dañadas y tomamos una que este bien para usarla como muestra
fd.run(s3_img_path, threshold= 0.5, overwrite= True, high_accuracy= True)

# Obtenemos los similarity
similarity = fd.similarity()

In [ ]:
invalid_img = [os.path.join("trash/fastdup/tmp/", path.replace("s3://hydrahi4ai/", "")) for path in fd.invalid_instances()["filename"].to_list()]
test_path = set(["/".join(path.split("/")[:-1]) for path in invalid_img])

In [ ]:
similarity.head(1)

In [ ]:
# lista de los archivos dañados, para descargarlos
for path in invalid_img:
    add_metadata(path)

In [ ]:
fd.run("trash/fastdup/tmp/ajio-myntra/alternative/20231219/", threshold= 0.5, overwrite= True, high_accuracy= True)